In [23]:
import requests
import json
import pandas as pd
import time
from dotenv import load_dotenv
import os
import logging
from config import setup_logging

In [24]:
load_dotenv()
API_KEY = os.getenv('API_KEY')
logger = logging.getLogger('api')

In [25]:
url = 'https://v3.football.api-sports.io/status'

headers = {
    'x-apisports-key': API_KEY,
}

response = requests.get(url, headers=headers)
logger.info(f"Статус ответа: {response.status_code}")
# response.json()

2025-11-12 12:57:02,654 - api - INFO - Статус ответа: 200


In [ ]:
def call_api(endpoint, params = {}):
    url = f"https://v3.football.api-sports.io/{endpoint}"
    headers = {
        "x-rapidapi-key": API_KEY
    }
    try:
        logger.info("Обращение к API")
        response = requests.get(
            url,
            params=params,
            headers=headers
        )
        logger.info(f"Статус ответа API: {response.status_code}")
        return response.json()
    except requests.exceptions.RequestException as e:
        logger.error(f"Ошибка при вызове API: {e}")
        return None

In [5]:
leagues = call_api('leagues')

In [6]:
leagues['response'][:2]

[{'league': {'id': 4,
   'name': 'Euro Championship',
   'type': 'Cup',
   'logo': 'https://media.api-sports.io/football/leagues/4.png'},
  'country': {'name': 'World', 'code': None, 'flag': None},
  'seasons': [{'year': 2008,
    'start': '2008-06-07',
    'end': '2008-06-29',
    'current': False,
    'coverage': {'fixtures': {'events': True,
      'lineups': True,
      'statistics_fixtures': False,
      'statistics_players': False},
     'standings': False,
     'players': True,
     'top_scorers': True,
     'top_assists': True,
     'top_cards': True,
     'injuries': False,
     'predictions': True,
     'odds': False}},
   {'year': 2012,
    'start': '2012-06-08',
    'end': '2012-07-01',
    'current': False,
    'coverage': {'fixtures': {'events': True,
      'lineups': True,
      'statistics_fixtures': False,
      'statistics_players': False},
     'standings': False,
     'players': True,
     'top_scorers': True,
     'top_assists': True,
     'top_cards': True,
     'i

In [7]:
def clean_leagues_columns(leagues_df):
    leagues_df['league_id'] = leagues_df['league'].apply(lambda d: d['id'])
    leagues_df['league_name'] = leagues_df['league'].apply(lambda d: d['name'])
    leagues_df['league_country'] = leagues_df['country'].apply(lambda d: d['name'])
    return leagues_df[['league_id', 'league_name', 'league_country']]

In [8]:
leagues_df = pd.DataFrame(leagues['response'])
leagues_df_cleaned = clean_leagues_columns(leagues_df)
leagues_sorted = leagues_df_cleaned.sort_values('league_id')
leagues_sorted.head()

,league_id,league_name,league_country
14,1,World Cup,World
116,2,UEFA Champions League,World
179,3,UEFA Europa League,World
0,4,Euro Championship,World
395,5,UEFA Nations League,World


In [9]:
leagues_sorted.to_csv('Leagues_list', index=False)

In [10]:
leagues_list = [39, 40, 41, 42, 140, 135, 136, 78, 79, 61, 62, 94, 253, 235, 71, 119, 128, 262, 218, 113, 239, 307, 62, 145, 210, 88]
most_popular_leagues = leagues_sorted[leagues_sorted['league_id'].isin(leagues_list)]
most_popular_leagues

,league_id,league_name,league_country
6,39,Premier League,England
19,40,Championship,England
25,41,League One,England
26,42,League Two,England
3,61,Ligue 1,France
2,62,Ligue 2,France
5,71,Serie A,Brazil
7,78,Bundesliga,Germany
112,79,2. Bundesliga,Germany
9,88,Eredivisie,Netherlands


In [ ]:
def players_data(league, season, page = 1, players_data_list = None):
    if players_data_list is None:
        players_data_list = []
    players = call_api('players', {'league': league, 'season': season, 'page': page})
    if 'response' in players:
        players_ids = []
        for player_data in players['response']:
            if 'player' in player_data and 'id' in player_data['player']:
                player_id = player_data['player']['id']
                players_ids.append(str(player_id))
        player_ids_str = "-".join(players_ids)
        players_trophies = call_api('trophies', {'players': player_ids_str})
        for i in range(len(players['response'])):
            for j in range(len(players_trophies['response'])):
                if players['response'][i]['player']['id'] == players_trophies['response'][j]['id']:
                    players['response'][i]['trophies'] = players_trophies['response'][j]['trophies']
        
        players_sidelined_reason_list = call_api('sidelined', {'players': player_ids_str})
        for i in range(len(players['response'])):
            for j in range(len(players_sidelined_reason_list['response'])):
                if players['response'][i]['player']['id'] == players_sidelined_reason_list['response'][j]['id']:
                    players['response'][i]['sidelined'] = players_sidelined_reason_list['response'][j]['sidelined']

        players_data_list.extend(players['response'])

        logger.info(f"Проверка на наличие следующе страницы. Текущая: {page}")
        paging = players.get('paging', {})
        current_page = paging.get('current', 1)
        total_pages = paging.get('total', 1)
        if current_page < total_pages:
            next_page = current_page + 1
            logger.info(f"Следующая страница существует. Текущая: {current_page}. Следующая: {next_page}")
            if next_page % 2 == 1:
                time.sleep(1)
                logger.info("Нечетная страница - приостановили программу.")
            logger.info(f"Запускаем сбор игроков для следующей страницы: {next_page}")
            players_data_list = players_data(league, season, next_page, players_data_list)

    
    return players_data_list

players = []
for league in leagues_list:
    logger.info(f"Загрузка игроков для лиги: {league}")
    players_league = players_data(league, 2024)
    logger.info(f"Загрузка игроков для лиги завершена: {league}")
    players.extend(players_league)
logger.info(f"Суммарное количество игроков по лигам: {len(players)}")

24098


In [13]:
def clean_players_stats_df(players):
    players_stats_df = pd.DataFrame(players)
    players_stats_df['player_id'] = players['player'].apply(lambda d: d['id'])
    players_stats_df['player_name'] = players['player'].apply(lambda d: d['name'])
    players_stats_df['age'] = players['player'].apply(lambda d: d['age'])
    players_stats_df['country_birth'] = players['player'].apply(lambda d: d['birth']['country'])
    players_stats_df['nationality'] = players['player'].apply(lambda d: d['nationality'])
    players_stats_df['player_height'] = players['player'].apply(lambda d: d['height'])
    players_stats_df['player_weight'] = players['player'].apply(lambda d: d['weight'])
    players_stats_df['player_photo'] = players['player'].apply(lambda d: d['photo'])
    players_stats_df['injured'] = players['player'].apply(lambda d: d['injured'])
    players_stats_df['team_id'] = players['statistics'].apply(lambda stats: [team_dict['team']['id'] for team_dict in stats][0])
    # players_stats_df['team'] = players['statistics'].apply(lambda stats: ', '.join([team_dict['team']['name'] for team_dict in stats]))
    players_stats_df['league'] = players['statistics'].apply(lambda d: d[0]['league']['country'] + ' ' + d[0]['league']['name'])
    players_stats_df['season'] = players['statistics'].apply(lambda d: d[0]['league']['season'])
    players_stats_df['position'] = players['statistics'].apply(lambda d: d[0]['games']['position'])
    players_stats_df['is_captain'] = players['statistics'].apply(lambda d: d[0]['games']['captain'])
    players_stats_df['shots_on'] = players['statistics'].apply(lambda d: d[0]['shots']['on'])
    players_stats_df['shots_total'] = players['statistics'].apply(lambda d: d[0]['shots']['total'])
    players_stats_df['goals'] = players['statistics'].apply(lambda d: d[0]['goals']['total'])
    players_stats_df['assists'] = players['statistics'].apply(lambda d: d[0]['goals']['assists'])
    players_stats_df['passes_total'] = players['statistics'].apply(lambda d: d[0]['passes']['total'])
    players_stats_df['passes_key'] = players['statistics'].apply(lambda d: d[0]['passes']['key'])
    players_stats_df['duels_total'] = players['statistics'].apply(lambda d: d[0]['duels']['total'])
    players_stats_df['duels_won'] = players['statistics'].apply(lambda d: d[0]['duels']['won'])
    players_stats_df['dribbles_attempts'] = players['statistics'].apply(lambda d: d[0]['dribbles']['attempts'])
    players_stats_df['dribbles_success'] = players['statistics'].apply(lambda d: d[0]['dribbles']['success'])
    players_stats_df['fouls'] = players['statistics'].apply(lambda d: d[0]['fouls']['drawn'])
    players_stats_df['yellow_cards'] = players['statistics'].apply(lambda d: d[0]['cards']['yellow'])
    players_stats_df['yellow_red_cards'] = players['statistics'].apply(lambda d: d[0]['cards']['yellowred'])
    players_stats_df['red_cards'] = players['statistics'].apply(lambda d: d[0]['cards']['red'])
    players_stats_df['sidelined_times'] = players['sidelined'].apply(lambda d: len(d) if isinstance(d, list) else 0)
    players_stats_df['trophies_count'] = players['trophies'].apply(lambda d: len(d) if isinstance(d, list) else 0)
    players_stats_df['player_rating'] = players['statistics'].apply(lambda d: d[0]['games']['rating'])
    return players_stats_df.iloc[:, 4:]

In [14]:
players_stats_df = pd.DataFrame(players)
cleaned_players_stats_df = clean_players_stats_df(players_stats_df)
cleaned_players_stats_df

,player_id,player_name,age,country_birth,nationality,player_height,player_weight,player_photo,injured,team_id,...,duels_won,dribbles_attempts,dribbles_success,fouls,yellow_cards,yellow_red_cards,red_cards,sidelined_times,trophies_count,player_rating
0,83,A. Danjuma,28.0,Nigeria,Netherlands,178,74,https://media.api-sports.io/football/players/8...,False,45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,8,None
1,105,F. Ballo-Touré,28.0,France,Senegal,182,70,https://media.api-sports.io/football/players/1...,False,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,14,None
2,172,D. Alli,29.0,England,England,188,76,https://media.api-sports.io/football/players/1...,False,45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,12,None
3,2699,S. Ghoddos,32.0,Sweden,Iran,175,79,https://media.api-sports.io/football/players/2...,False,55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,2,None
4,18765,André Gomes,32.0,Portugal,Portugal,188,84,https://media.api-sports.io/football/players/1...,False,45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36,35,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24093,1815,J. de Guzmán,38.0,Canada,Netherlands,173,70,https://media.api-sports.io/football/players/1...,False,426,...,2.0,NaN,NaN,1.0,1.0,NaN,0.0,11,14,6.450000
24094,2109,S. Bastien,29.0,Belgium,Congo DR,175,75,https://media.api-sports.io/football/players/2...,False,205,...,45.0,10.0,8.0,12.0,0.0,0.0,0.0,5,9,6.731818
24095,25321,D. Klaassen,32.0,Netherlands,Netherlands,179,70,https://media.api-sports.io/football/players/2...,False,194,...,90.0,17.0,7.0,20.0,0.0,0.0,0.0,11,30,7.058064
24096,138874,A. Tsoungui,23.0,Belgium,Belgium,185,68,https://media.api-sports.io/football/players/1...,False,209,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1,2,None


In [ ]:
teams = []
for league in leagues_list:
    logger.info(f"Загрузка команд для лиги: {league}")
    teams_league = call_api('teams', {'league': league, 'season': 2024})
    logger.info(f"Загрузка комманд для лиги завершена: {league}")
    teams.extend(teams_league['response'])
logger.info(f"Суммарное количество комманд по лигам: {len(teams)}")
teams[:3]

[{'team': {'id': 33,
   'name': 'Manchester United',
   'code': 'MUN',
   'country': 'England',
   'founded': 1878,
   'national': False,
   'logo': 'https://media.api-sports.io/football/teams/33.png'},
  'venue': {'id': 556,
   'name': 'Old Trafford',
   'address': 'Sir Matt Busby Way',
   'city': 'Manchester',
   'capacity': 76212,
   'surface': 'grass',
   'image': 'https://media.api-sports.io/football/venues/556.png'}},
 {'team': {'id': 34,
   'name': 'Newcastle',
   'code': 'NEW',
   'country': 'England',
   'founded': 1892,
   'national': False,
   'logo': 'https://media.api-sports.io/football/teams/34.png'},
  'venue': {'id': 562,
   'name': "St. James' Park",
   'address': 'St. James&apos; Street',
   'city': 'Newcastle upon Tyne',
   'capacity': 52758,
   'surface': 'grass',
   'image': 'https://media.api-sports.io/football/venues/562.png'}},
 {'team': {'id': 35,
   'name': 'Bournemouth',
   'code': 'BOU',
   'country': 'England',
   'founded': 1899,
   'national': False,
   '

In [16]:
def clean_teams_df(teams):
    teams_df = pd.DataFrame(teams)
    teams_df['team_id'] = teams_df['team'].apply(lambda d: d['id'])
    teams_df['team_name'] = teams_df['team'].apply(lambda d: d['name'])
    teams_df['team_country'] = teams_df['team'].apply(lambda d: d['country'])
    teams_df['team_logo'] = teams_df['team'].apply(lambda d: d['logo'])
    return teams_df[['team_id', 'team_name', 'team_country', 'team_logo']]

In [17]:
cleaned_teams_df = clean_teams_df(teams)
cleaned_teams_df

,team_id,team_name,team_country,team_logo
0,33,Manchester United,England,https://media.api-sports.io/football/teams/33.png
1,34,Newcastle,England,https://media.api-sports.io/football/teams/34.png
2,35,Bournemouth,England,https://media.api-sports.io/football/teams/35.png
3,36,Fulham,England,https://media.api-sports.io/football/teams/36.png
4,39,Wolves,England,https://media.api-sports.io/football/teams/39.png
...,...,...,...,...
503,419,Almere City FC,Netherlands,https://media.api-sports.io/football/teams/419...
504,420,Cambuur,Netherlands,https://media.api-sports.io/football/teams/420...
505,421,Den Bosch,Netherlands,https://media.api-sports.io/football/teams/421...
506,426,Sparta Rotterdam,Netherlands,https://media.api-sports.io/football/teams/426...


In [18]:
cleaned_teams_df = cleaned_teams_df.set_index('team_id')
cleaned_players_stats_df = cleaned_players_stats_df.set_index('team_id')

In [19]:
cleaned_players_stats_df = pd.merge(cleaned_players_stats_df, cleaned_teams_df, left_index=True, right_index=True, how='left')
cleaned_players_stats_df = cleaned_players_stats_df.reset_index()
cleaned_players_stats_df

,team_id,player_id,player_name,age,country_birth,nationality,player_height,player_weight,player_photo,injured,...,fouls,yellow_cards,yellow_red_cards,red_cards,sidelined_times,trophies_count,player_rating,team_name,team_country,team_logo
0,45,83,A. Danjuma,28.0,Nigeria,Netherlands,178,74,https://media.api-sports.io/football/players/8...,False,...,NaN,NaN,NaN,NaN,11,8,None,Everton,England,https://media.api-sports.io/football/teams/45.png
1,36,105,F. Ballo-Touré,28.0,France,Senegal,182,70,https://media.api-sports.io/football/players/1...,False,...,NaN,NaN,NaN,NaN,4,14,None,Fulham,England,https://media.api-sports.io/football/teams/36.png
2,45,172,D. Alli,29.0,England,England,188,76,https://media.api-sports.io/football/players/1...,False,...,NaN,NaN,NaN,NaN,17,12,None,Everton,England,https://media.api-sports.io/football/teams/45.png
3,55,2699,S. Ghoddos,32.0,Sweden,Iran,175,79,https://media.api-sports.io/football/players/2...,False,...,NaN,NaN,NaN,NaN,5,2,None,Brentford,England,https://media.api-sports.io/football/teams/55.png
4,45,18765,André Gomes,32.0,Portugal,Portugal,188,84,https://media.api-sports.io/football/players/1...,False,...,NaN,NaN,NaN,NaN,36,35,None,Everton,England,https://media.api-sports.io/football/teams/45.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25916,426,1815,J. de Guzmán,38.0,Canada,Netherlands,173,70,https://media.api-sports.io/football/players/1...,False,...,1.0,1.0,NaN,0.0,11,14,6.450000,Sparta Rotterdam,Netherlands,https://media.api-sports.io/football/teams/426...
25917,205,2109,S. Bastien,29.0,Belgium,Congo DR,175,75,https://media.api-sports.io/football/players/2...,False,...,12.0,0.0,0.0,0.0,5,9,6.731818,Fortuna Sittard,Netherlands,https://media.api-sports.io/football/teams/205...
25918,194,25321,D. Klaassen,32.0,Netherlands,Netherlands,179,70,https://media.api-sports.io/football/players/2...,False,...,20.0,0.0,0.0,0.0,11,30,7.058064,Ajax,Netherlands,https://media.api-sports.io/football/teams/194...
25919,209,138874,A. Tsoungui,23.0,Belgium,Belgium,185,68,https://media.api-sports.io/football/players/1...,False,...,NaN,0.0,0.0,0.0,1,2,None,Feyenoord,Netherlands,https://media.api-sports.io/football/teams/209...


In [20]:
cleaned_players_stats_df.to_csv('players_stats.csv', index=False)